In [2]:
import pandas as pd # for dataframes manipulation
#import matplotlib.pyplot as plt

# Import the data
firms = pd.read_csv('SP_500_firms.csv')
close_prices = pd.read_csv('SP_500_close_2015.csv')

# Create subset of the original dataset which includes
# only 10 firms - used for test purposes
firms_small = firms.iloc[:10, :]
close_prices_small = close_prices.iloc[:, :11]

print(firms_small[:5])
print(close_prices_small[:5])

  Symbol                 Name                  Sector
0    MMM           3M Company             Industrials
1    ABT  Abbott Laboratories             Health Care
2   ABBV               AbbVie             Health Care
3    ACN        Accenture plc  Information Technology
4   ATVI  Activision Blizzard  Information Technology
         Date         MMM        ABT       ABBV        ACN       ATVI  \
0  2015-01-02  156.678596  43.160459  61.986410  86.129228  19.765196   
1  2015-01-05  153.145069  43.170070  60.819874  84.674997  19.490271   
2  2015-01-06  151.511999  42.679830  60.518833  84.064223  19.126976   
3  2015-01-07  152.610267  43.025880  62.964797  85.828689  18.714587   
4  2015-01-08  156.267949  43.910238  63.623323  87.137495  18.901144   

          AYI       ADBE         AAP        AES        AET  
0  139.234407  72.339996  158.132353  12.860543  87.354435  
1  135.889914  71.980003  156.047994  12.494440  86.173965  
2  134.187800  70.529999  155.938290  12.212822  86.30

In [4]:
# ### 1 ###
# Create a function that takes a dataframe with closing prices 
# and returns a dataframe with daily returns
# Date is to define whether the dataframe has a first
def StockReturns(cl_p, Date = True):
    """
    Input:  The dataframe with the prices of the stocks
            A logical argument that equals True if the first column
            in the dataframe represents dates, False otherwise
    Output: A dataframe with the daily returns of the stocks - same 
            number of columns as the input vector and one row less 
            than the input vector
    """    
    if Date:
        d_ret = pd.DataFrame(cl_p.iloc[1:, 0], columns=['Date'])
        j = 1
    else:
        d_ret = pd.DataFrame()
        j = 0
    for i in range(j, cl_p.shape[1]):
        d_ret[cl_p.columns[i]] = (cl_p.iloc[1:,i].values - cl_p.iloc[:-1,i].values) / cl_p.iloc[:-1,i].values
    return d_ret

In [5]:
# Test the previous function on the small dataset

# Daily returns including the Date column    
daily_returns = StockReturns(close_prices_small) 
print(daily_returns.iloc[:5, :5])

# Daily returns excluding the Date column
daily_returns_2 = StockReturns(close_prices_small.iloc[:, 1:], Date = False)
print(daily_returns_2.iloc[:5, :5])   

         Date       MMM       ABT      ABBV       ACN
1  2015-01-05 -0.022553  0.000223 -0.018819 -0.016884
2  2015-01-06 -0.010664 -0.011356 -0.004950 -0.007213
3  2015-01-07  0.007249  0.008108  0.040417  0.020989
4  2015-01-08  0.023967  0.020554  0.010459  0.015249
5  2015-01-09 -0.012284 -0.010508 -0.027355 -0.001113
        MMM       ABT      ABBV       ACN      ATVI
0 -0.022553  0.000223 -0.018819 -0.016884 -0.013910
1 -0.010664 -0.011356 -0.004950 -0.007213 -0.018640
2  0.007249  0.008108  0.040417  0.020989 -0.021561
3  0.023967  0.020554  0.010459  0.015249  0.009969
4 -0.012284 -0.010508 -0.027355 -0.001113 -0.018182


In [6]:
# ### 2 ###
# Create a function that returns a list of correlations between all the
# firms
def Correlations(d_ret):
    """
    Input:  A dataframe with the daily returns of the stocks - The 
            first column can either indicate dates or not
    Output: A list of tuples. Each tuple in the list have 3 elements:
            1. The correlation between two firms
            2 and 3. The firms for which we compute the correlation
    """
    cor = d_ret.corr()
    n = int(cor.shape[0])
    cor_list = []
    for i in range(1, n):
        for j in range(0, i):
            cor_list.append((cor.iloc[i, j], cor.columns.values[i], cor.columns.values[j]))
    return cor_list

In [9]:
# Test the previous function on the small dataset

# Correlation list
correlation_list = Correlations(daily_returns)
print(correlation_list[:5])

[(0.59866616402973749, 'ABT', 'MMM'), (0.32263699601940204, 'ABBV', 'MMM'), (0.48366885347180477, 'ABBV', 'ABT'), (0.63205934885601844, 'ACN', 'MMM'), (0.64408052989752662, 'ACN', 'ABT')]


In [11]:
# Create a function that sorts the list based on the correlation in 
# decreasing order
def SortCorrs(cor_list): 
    """
    Input:  A list of tuples. Each tuple in the list have 3 elements:
            1. The correlation between two firms
            2 and 3. The firms for which we compute the correlation
    Output: Returns the same list of tuples ordered based on the
            first element of the tuples, e.g. the correlation
    """
    return sorted(cor_list, reverse = True)

In [12]:
# Test the previous function

ordered_list = SortCorrs(correlation_list)
print(ordered_list[:5])

[(0.64408052989752662, 'ACN', 'ABT'), (0.63205934885601844, 'ACN', 'MMM'), (0.59866616402973749, 'ABT', 'MMM'), (0.58567123295127221, 'ADBE', 'ABT'), (0.56075923030727226, 'ADBE', 'ACN')]


In [13]:
# ### 3 ###
# Create the function that performs the clustering algorithm
def ClusteringAlg(ord_list, k = 0):
    """
    Input:  The ordered list of tuples which include the
            correlations between firms and the firms themselves
            The number of iterations for the clustering algorithm
    Output: A list of lists which represent the individual sets 
            or clusters
    """
    # Initialize the list of the sets. Each sets is represented by a
    # list which includes only one argument, e.g. each firm is 
    # represented as a cluster by itself
    sets = []
    for i in range(len(ordered_list)):
        if not([ord_list[i][1]] in sets):
            sets.append([ord_list[i][1]])
        if not([ord_list[i][2]] in sets):
            sets.append([ord_list[i][2]]) 
    
    # Repeat the algorithm k times:
    # Finds in which sets are the firms with the current highest
    # correlation and merges those two lists into one, which insludes
    # both the firms
    for j in range(k):
        nd1 = ord_list[j][1]
        nd2 = ord_list[j][2]
        fl1, fl2 = False, False    
        for i in range(len(sets)):
            if (nd1 in sets[i]):
                idx1 = i
                fl1 = True
            elif (nd2 in sets[i]):
                idx2 = i
                fl2 = True
        if (fl1 and fl2):
            sets[idx1] = sets[idx1] + sets[idx2]
            sets.remove(sets[idx2])
    return sets

In [14]:
# Test the previous function in the small dataset

clusters = ClusteringAlg(ordered_list, 5)
print(clusters)

[['ADBE', 'ACN', 'ABT', 'MMM'], ['AYI'], ['ABBV'], ['ATVI'], ['AET'], ['AES'], ['AAP']]
